### Notebook to create a CalibratedImage object test methods

Import statements

In [ ]:
import os
import numpy as np
from astropy.io import fits
import pylab as plt
import logging
from ConfigParser import ConfigParser

Choose a PTF image file and a SExtractor catalog file to create a image object. Here there is a sample PTF image obtained from the IRSA website (http://irsa.ipac.caltech.edu/applications/ptf/) along with the SExtractor catalog file.

In [ ]:
image_file = 'etc/PTF_200903171534_i_p_scie_t034051_u012218978_f02_p100037_c02.fits'
se_cat_file= 'etc/PTF_200903171534_c_p_scie_t034051_u012218978_f02_p100037_c02.ctlg'

The image looks like the following

In [ ]:
%matplotlib inline
image_data = fits.getdata(image_file)
plt.imshow(image_data, cmap='gray')
#plt.show()

- Create a logger object to write logs into. By default it is created in the *etc* directory.
- Also point to a config file. The script write_config.py will write one in the *etc* directory

In [ ]:
run config_writer.py

In [ ]:
#log_fmt = '%(asctime) -- %(levelname) : %(message)'
logfile = os.path.join(os.getcwdu(), 'etc/fakes.log')
confile = os.path.join(os.getcwdu(), 'etc/conf.ini')
logging.basicConfig(filename = logfile, filemode = 'w', level = logging.DEBUG)
logger  = logging.getLogger()
config  = ConfigParser()
config.read(confile)

The logging.basicConfig(...) might give a deprecation warning for Python3 about the way the filename is passed. It asks to supply unicode strings. But this is fine for us.

#### We can create the CalibratedImage object

In [ ]:
from image import CalibratedImage

img_obj = CalibratedImage(image_file, se_cat_file, config=config, logger=logger)

#### Test out the methods
- There are many method names ending in _Check_. Each returns a boolean array (a mask) which can be used anytime later to pull out the data from the SExtractor catalog.
- Each check reads for a max and min value from the config file, which are supposed to be the upper and lower cuts  of the check. For example _starElgnCheck()_ filters the objects based on the cuts applied.
- If the upper and lower values are not supplied (no value in the config file), it is assumed that bound is not to be supplied (see config file).
- Stars and Galaxies have separate checks, since the constraint put on each might be different.

In [ ]:
check1 = img_obj.starFWHMCheck()
check2 = img_obj.galMagCheck()
print check1
print np.sum(check2)

#### Perform all the checks on the stars and print the filtered SE catalog


In [ ]:
img_obj.giveStarsTable()

In [ ]:
img_obj.giveGalaxiesTable()

### Clone Stamped injection

Clone stamping takes one part of the image and _stamp_ it into another part. For example, pick out a bright star in an image and put it in a galaxy. 
- We need to specify a grid, say 20x20 pixels, that will be stamped.
- Need to subtract the backgrouond from the source location
- Scale the image i.e. make the source brighter or dimmer (maybe change the PSF in the future)
- Add it to the target location.

In [ ]:
from image import CloneStampedInjection

clone_image_obj = CloneStampedInjection(image_file, se_cat_file, num_inj=3, config=config, logger=logger)

We need to supply how many injections we want to make, using the parameter *num\_inj*. For a clone stamped image, *num\_inj* number of stars will be injected in *num\_inj* number of galaxies. One of the scenario is to stamp bright stars in the field into dim galaxies.

In [ ]:
stamped_image = clone_image_obj.stampBrightsInDims()

The method _stampBrightsInDims_ will return the image data after the stamping. Based on a flag in the config file, it should be possible to either modify the original image data or return a duplicate stamped image without modifying the original.
Spotting the difference in image might be difficult visually. However, we know there is some difference between the original and stamped image

In [ ]:
np.sum( np.abs(stamped_image - clone_image_obj.img_data) != 0 )